<a href="https://colab.research.google.com/github/ujwalkr94/LLM_Developer/blob/main/notebooks/Crawl_a_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Crawl_a_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index==0.12.11 llama-index-llms-openai==0.3.13 openai==1.59.8 google-generativeai==0.8.3 newspaper3k==0.2.8 lxml_html_clean==0.4.1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
USESCRAPER_API_KEY = "[USESCRAPER_API_KEY]"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY1')
# os.environ["USESCRAPER_API_KEY"] = userdata.get('USESCRAPER_API_KEY')

# USESCRAPER_API_KEY = userdata.get('USESCRAPER_API_KEY')

There are two primary methods for extracting webpage content. The first method involves having a list of URLs; one can iterate through this list to retrieve the content of each page. The second method, web crawling, requires using a script or service to extract page URLs from a sitemap or manually following links on the page to access all the content. Initially, we will explore web scraping techniques before discussing how to use a service like usescraper.com to perform web crawling.


# 1. Scraping using `newspaper` Library


## Define URLs


In [ ]:
urls = [
    "https://docs.llamaindex.ai/en/stable/understanding",
    "https://docs.llamaindex.ai/en/stable/understanding/using_llms/using_llms/",
    "https://docs.llamaindex.ai/en/stable/understanding/indexing/indexing/",
    "https://docs.llamaindex.ai/en/stable/understanding/querying/querying/",
]

## Get Page Contents


In [ ]:
import newspaper

pages_content = []

# Retrieve the Content
for url in urls:
    try:
        article = newspaper.Article(url)
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append(
                {"url": url, "title": article.title, "text": article.text}
            )
    except:
        continue

In [ ]:
pages_content[0]

{'url': 'https://docs.llamaindex.ai/en/stable/understanding',
 'title': 'Building an LLM Application',
 'text': "Building an LLM application#\n\nWelcome to the beginning of Understanding LlamaIndex. This is a series of short, bite-sized tutorials on every stage of building an LLM application to get you acquainted with how to use LlamaIndex before diving into more advanced and subtle strategies. If you're an experienced programmer new to LlamaIndex, this is the place to start.\n\nKey steps in building an LLM application#\n\nTip If you've already read our high-level concepts page you'll recognize several of these steps.\n\nThis tutorial has three main parts: Building a RAG pipeline, Building an agent, and Building Workflows, with some smaller sections before and after. Here's what to expect:\n\nUsing LLMs : hit the ground running by getting started working with LLMs. We'll show you how to use any of our dozens of supported LLMs, whether via remote API calls or running locally on your mac

In [ ]:
len(pages_content)

4

## Convert to Document


In [ ]:
from llama_index.core.schema import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(text=row["text"], metadata={"title": row["title"], "url": row["url"]})
    for row in pages_content
]

# 2. Submit the Crawler Job


In [ ]:
import requests
import json

urls_to_crawl = [
    "https://docs.llamaindex.ai/en/stable/understanding/", # add your URLs here, e.g. "https://docs.llamaindex.ai/en/stable/understanding/"
]

payload = {
    "urls": urls_to_crawl,  # list of urls to crawl
    "output_format": "markdown",  # text, html, markdown
    "output_expiry": 604800,  # Automatically delete after X seconds
    "min_length": 50,  # Skip pages with less than X characters
    "page_limit": 3,  # Maximum number of pages to crawl
    "force_crawling_mode": "link",  # "link" follows links in the page reccursively, or "sitemap" to find pages from website's sitemap
    "block_resources": True,  # skip loading images, stylesheets, or scripts
    "include_linked_files": False,  # include files (PDF, text, ...) in output
}
headers = {
    "Authorization": "Bearer " + USESCRAPER_API_KEY,
    "Content-Type": "application/json",
}

response = requests.request(
    "POST", "https://api.usescraper.com/crawler/jobs", json=payload, headers=headers
)

response = json.loads(response.text)

print(response)

{'org': '2492', 'id': '7YDDVX24F9WD1JMC1EE83PT88Q', 'urls': ['https://docs.llamaindex.ai/en/stable/understanding/'], 'exclude_globs': [], 'exclude_elements': 'nav, header, footer, script, style, noscript, svg, [role="alert"], [role="banner"], [role="dialog"], [role="alertdialog"], [role="region"][aria-label*="skip" i], [aria-modal="true"]', 'output_format': 'markdown', 'output_expiry': 604800, 'min_length': 50, 'page_limit': 3, 'force_crawling_mode': 'link', 'block_resources': True, 'include_linked_files': False, 'createdAt': 1737451564566, 'status': 'starting', 'use_browser': True, 'sitemapPageCount': 0, 'notices': []}


## Get the Status


In [ ]:
url = "https://api.usescraper.com/crawler/jobs/{}".format(response["id"])

status_res = requests.request("GET", url, headers=headers)

status_res = json.loads(status_res.text)

print(status_res["status"])
print(status_res["progress"])

succeeded
{'scraped': 3, 'discarded': 0, 'failed': 0}


## Get the Data


In [ ]:
url = "https://api.usescraper.com/crawler/jobs/{}/data".format(response["id"])

data_res = requests.request("GET", url, headers=headers)

data_res = json.loads(data_res.text)

print(data_res)

{'data': [{'meta': {'url': 'https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agents/', 'fetchedUrl': 'https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agents/', 'fetchedUrlStatusCode': 200, 'meta': {'title': 'Multi-agent workflows - LlamaIndex'}}, 'text': ' \n[ Skip to content ](https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agents/#multi-agent-workflows)\n#Multi-Agent Workflows[#](https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agents/#multi-agent-workflows)\nThe AgentWorkflow uses Workflow Agents to allow you to create a system of one or more agents that can collaborate and hand off tasks to each other based on their specialized capabilities. This enables building complex agent systems where different agents handle different aspects of a task.\n\nTip\n\nThe AgentWorkflow class is built on top of LlamaIndex Workflows. For more information on how workflows work, check out the [detailed guide](https://docs.llamaindex.ai/en/stabl

In [ ]:
print("URL:", data_res["data"][0]["meta"]["url"])
print("Title:", data_res["data"][0]["meta"]["meta"]["title"])
print("Content:", data_res["data"][0]["text"][0:500], "...")

URL: https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agents/
Title: Multi-agent workflows - LlamaIndex
Content:  
[ Skip to content ](https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agents/#multi-agent-workflows)
#Multi-Agent Workflows[#](https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agents/#multi-agent-workflows)
The AgentWorkflow uses Workflow Agents to allow you to create a system of one or more agents that can collaborate and hand off tasks to each other based on their specialized capabilities. This enables building complex agent systems where different agents handle di ...


## Convert to Document


In [ ]:
from llama_index.core.schema import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(
        text=row["text"],
        metadata={"title": row["meta"]["meta"]["title"], "url": row["meta"]["url"]},
    )
    for row in data_res["data"]
]

# Create RAG Pipeline


In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(temperature=0, model="gpt-4o-mini")

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=30)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
res = query_engine.query("What is a query engine?")

In [ ]:
res.response

'A query engine is a component that processes and executes queries against a data source, retrieving relevant information based on specified criteria. It typically works in conjunction with indexing strategies to enhance the relevance, speed, and accuracy of the data retrieval process.'

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("URL\t", src.metadata["url"])
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 ae9c2540-f0f0-44b8-aa78-615a944f547b
Title	 Building an LLM Application - LlamaIndex
URL	 https://docs.llamaindex.ai/en/stable/understanding/
Score	 0.3028895128495175
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 2a4b3a83-8904-4653-9472-5f28db53d30a
Title	 Building an LLM Application - LlamaIndex
URL	 https://docs.llamaindex.ai/en/stable/understanding/
Score	 0.24641542534057687
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
